1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
  
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서로 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install langchain-chroma
%pip install -U langchain langchain-core langchainhub


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## chunk_size = 청크 하나를 구성하는 최대 토큰 수
## chunk_overlap = 청크들이 겹치는 토큰 수
## 예를 들어, chunk_size=1500, chunk_overlap=200이면
## 각 청크는 최대 1500 토큰까지 포함할 수 있으며,
## 각 청크는 이전 청크와 200 토큰이 겹치게 됩니다.
## 이 설정은 문서의 연속성을 유지하면서도
## 청크 간의 연결성을 높이는 데 도움이 됩니다.
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500, 
  chunk_overlap=200
  )

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

ModuleNotFoundError: No module named 'langchain.text_splitter'

In [5]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")


In [6]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory="./chroma",
#     collection_name="chroma_tax_docs"
# )

database = Chroma(
    persist_directory="./chroma",
    embedding_function=embedding,
    collection_name="chroma_tax_docs"
)

In [7]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query = query, k=3)


In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [24]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

[User Question]
{query}
"""

In [26]:
ai_response = llm.invoke(prompt)
ai_response.content

"한국의 소득세는 과세표준에 따라 누진세율이 적용되며, 기본적으로 과세 표준에 도달한 금액에 대해 해당 세율만큼의 세금이 부과됩니다. 하지만 부가적인 공제와 세액감면이 다양하기 때문에, 정확한 세금을 계산하기 위해서는 더 많은 정보가 필요합니다.\n\n소득세율은 대략 다음과 같습니다 (2023년 기준):\n- 12%: 1,200만 원 초과 ~ 4,600만 원 이하\n- 15%: 4,600만 원 초과 ~ 8,800만 원 이하\n\n5,000만 원의 연봉에 대한 간단한 소득세 계산을 예로 들어보겠습니다. 이는 기본적인 '세전' 계산이며, 다양한 공제 및 추가 요인이 반영되지 않았다는 점을 유의해 주세요.\n\n1. 0원 ~ 1,200만원: 6%\n2. 1,200만원 ~ 4,600만원: 15%\n3. 4,600만원 초과 ~ 5,000만원: 24%\n\n계산 과정:\n- 0원 ~ 1,200만원: 1,200만원 × 6% = 72만원\n- 1,200만원 ~ 4,600만원: 3,400만원 × 15% = 510만원\n- 4,600만원 ~ 5,000만원: 400만원 × 24% = 96만원\n\n총 소득세 = 72만원 + 510만원 + 96만원 = 678만원\n\n다만, 이는 기본 계산 방식일 뿐이고 구체적인 소득세액은 개인의 구체적인 상황 (예: 공제 항목, 보험료 지급액 등)에 따라 달라질 수 있으므로 정확한 금액을 확인하기 위해서는 국세청의 공식 계산기를 사용하는 것이 좋습니다."

In [9]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

combine_docs_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(database.as_retriever(), combine_docs_chain)

rag_chain.invoke({"question": query, "context": some_retrieved_context})


AttributeError: 'function' object has no attribute 'from_llm_and_tools'